In [1]:
import os
import copy
import argparse
import warnings
from tqdm import tqdm
import pandas as pd
import numpy as np
import glob
from ketos.audio.audio_loader import  AudioFrameLoader, FrameStepper, audio_repres_dict
from ketos.neural_networks import load_model_file
from ketos.neural_networks.resnet import ResNetInterface
from ketos.neural_networks.dev_utils.detection import process, process_audio_loader, save_detections, merge_overlapping_detections

2021-12-02 03:19:44.117431: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-02 03:19:44.532955: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10477 MB memory:  -> device: 0, name: NVIDIA TITAN V, pci bus id: 0000:a1:00.0, compute capability: 7.0
/home/sadman/anaconda3/envs/deeplearning/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [2]:
# All the paths are indicating the location at the PC at DAL!
data_dir='/data/WCS/' # Path to the folder that contains all the audio files

model='/home/sadman/arctic_mammal/trained_models/bh_detector_v02.kt'
output_file_short_note='_bh_detector_v02_' # add anything here to add in the name of the detections file! By default keep it an empty string

root_path='/home/sadman/arctic_mammal/' # Path where this program is stored!

detection_save_path=root_path+'results/model_detections/'
annot_file_path=root_path+'annotations/test/'

num_segs=128
step_size=None
buffer=0.0
win_len=1
threshold=0.0
group=False
progress_bar=True
merge=False

save_detections_flag=True

In [3]:
# load the classifier and the spectrogram parameters
model, audio_repr = load_model_file(model, './tmp_folder', load_audio_repr=True)
spec_config = audio_repr[0]['spectrogram']
spec_config

{'type': 'MagSpectrogram',
 'rate': 10000,
 'window': 0.051,
 'step': 0.01955,
 'freq_min': 0,
 'freq_max': 5000,
 'window_func': 'hamming',
 'duration': 3.0,
 'normalize_wav': True,
 'transforms': [{'name': 'reduce_tonal_noise'},
  {'name': 'normalize', 'mean': 0.0, 'std': 1.0}]}

In [4]:
def save_detections(detections, save_to):
    """ Save the detections to a csv file

        Args:
            detections: numpy.array
                List of detections
            save_to:string
                The path to the .csv file where the detections will be saved.
                Example: "/home/user/detections.csv"
    """
    if len(detections) == 0: return

    a = np.array(detections)
    df = pd.DataFrame({'filename':a[:,0], 'start':a[:,1], 'duration':a[:,2], 'score':a[:,3]})
    include_header = not os.path.exists(save_to)
    df.to_csv(save_to, mode='a', index=False, header=include_header)
    return df

# Annotation file pre-processing

In [5]:
def preprocess_annot(annot_df, delete_columns=['Selection', 'View']):
    """ Delete unwanted columns from the annotations dataframe and 
        apply necessary pre-processing on the annotation columns
     
        Args:
            annot_df: pandas DataFrame
                Annotation table.
            delete_columns: list
                List of columns to delete from the annotations dataframe
                default values ('Selection', 'View')

        Returns:
            annot_df: pandas DataFrame
                Annotation table after pre-processing

    """
    # Delete unnecessary columns
    for column in delete_columns:
        del annot_df[column]
        
    annot_df.rename({'Begin Path': 'filename',
                    'File Offset (s)': 'start',
                    'species': 'label'}, axis='columns', inplace =True)

    # Modify filepath (Discard the drive location (e.g., D:/))
    annot_df['filename']=annot_df['filename'].apply(lambda x: x[3:len(x)]) 
    # Modify filepath to replace \ with / in the filename
    annot_df['filename']=annot_df['filename'].apply(lambda x: x.replace("\\", "/")) 
    # Modify filepath to add the data root dir (/data/WCS/)
    annot_df['filename']=annot_df['filename'].apply(lambda x: data_dir+x)
    # Calculate End time
    annot_df['end']=annot_df['start']+annot_df['Delta Time (s)']
    return annot_df

In [7]:
# load each test file and do all the calculations on them
for file_path in glob.glob(annot_file_path+'*.csv'):
    annot_df = pd.read_csv(file_path)
    annot_df=preprocess_annot(annot_df)
    file_list = list(annot_df['filename'].unique())
    
    # extract the audio folder path
    last_index_of_slash=file_list[0].rindex('/')
    audio_folder=file_list[0][0:last_index_of_slash]
        
    # extract the file name from the folder path in the format of "filename.csv"
    file_name=file_path[file_path.rindex('/')+1:len(file_path)]
    print("file_name", file_name)
    
    # initialize the audio loader
    audio_loader = AudioFrameLoader(frame=spec_config['duration'], 
                                    step=step_size, 
                                    path=audio_folder, 
                                    filename=file_list, 
                                    repres=spec_config)

    # process the audio data
    detections = process_audio_loader(audio_loader, 
                                      model=model, 
                                      batch_size=num_segs, 
                                      buffer=buffer, 
                                      threshold=threshold, 
                                      group=group, 
                                      win_len=win_len, 
                                      progress_bar=progress_bar)
    if merge == True:
        detections = merge_overlapping_detections(detections)

    if save_detections_flag == True:
        # get the output file name
        if(len(output_file_short_note)==0):
            output=detection_save_path+"detections_"+file_name
        else:
            output=detection_save_path+"detections"+output_file_short_note+file_name

        # save the each detections on test dataset
        if os.path.isfile(output): os.remove(output) #remove, if already exists
        print(f'{len(detections)} detections saved to {output}')
        detections_df=save_detections(detections=detections, save_to=output)

file_name CB50_test.csv


  0%|                                                   | 0/728 [00:00<?, ?it/s]2021-12-02 03:22:10.857909: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-02 03:22:11.665566: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201
100%|█████████████████████████████████████████| 728/728 [24:59<00:00,  2.06s/it]


93200 detections saved to /home/sadman/arctic_mammal/results/model_detections/detections_bh_detector_v02_CB50_test.csv
file_name CB300_test.csv


100%|█████████████████████████████████████████| 410/410 [13:59<00:00,  2.05s/it]


52600 detections saved to /home/sadman/arctic_mammal/results/model_detections/detections_bh_detector_v02_CB300_test.csv
